# Unsteady Heat Conduction Equation

$$\frac{\partial T(x,t)}{\partial t} = \alpha \frac{\partial^2 T(x,t)}{\partial x^2}$$

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Problem parameters
L = 1.0  # Length of the domain
alpha = 1.0  # Thermal diffusivity

# Discretization parameters
Nx = 100  # Number of spatial grid points
Nt = 100  # Number of time steps
dx = L / (Nx - 1)
max_time = 0.2  # New maximum time
dt = max_time / (Nt - 1)

# Initial condition
def T_initial(x):
    return np.sin(np.pi * x) + 0.5 * np.sin(3 * np.pi * x) + 0.25 * np.sin(5 * np.pi * x)

# Create a spatial grid
x_values = np.linspace(0, 1, 100)

# Evaluate the initial condition at the grid points
T_values = T_initial(x_values)

# Plot the initial temperature distribution
plt.figure()
plt.plot(x_values, T_values, label="Initial Temperature")
plt.xlabel("x")
plt.ylabel("Temperature")
plt.title("Initial Temperature Distribution (t=0)")
plt.legend()
plt.show()

# Boundary conditions (Dirichlet)
T_left = 0.0
T_right = 0.0

## Crank-Nicolson (Finite Difference Method)

In [ ]:
# Initialize the temperature matrix
T_matrix = np.zeros((Nx, Nt))

# Set the initial condition
T_matrix[:, 0] = T_initial(np.linspace(0, L, Nx))

# Set up the tridiagonal matrix for the Crank-Nicolson method
A = np.eye(Nx - 2) * (2 + 2 * alpha * dt / dx**2) + np.eye(Nx - 2, k=1) * (-alpha * dt / dx**2) + np.eye(Nx - 2, k=-1) * (-alpha * dt / dx**2)
B = np.eye(Nx - 2) * (2 - 2 * alpha * dt / dx**2) + np.eye(Nx - 2, k=1) * (alpha * dt / dx**2) + np.eye(Nx - 2, k=-1) * (alpha * dt / dx**2)

# Time-stepping loop
for n in range(1, Nt):
    # Set up the right-hand side vector
    rhs = np.dot(B, T_matrix[1:-1, n - 1])
    rhs[0] += alpha * dt / dx**2 * T_left
    rhs[-1] += alpha * dt / dx**2 * T_right

    # Solve the linear system
    T_matrix[1:-1, n] = np.linalg.solve(A, rhs)

    # Enforce boundary conditions
    T_matrix[0, n] = T_left
    T_matrix[-1, n] = T_right

In [ ]:
plt.figure()
plt.contourf(np.linspace(0, L, Nx), np.linspace(0, max_time, Nt), T_matrix.T, cmap="jet")
plt.xlabel("x")
plt.ylabel("t")
plt.title("Temperature Distribution (Crank-Nicolson)")
plt.colorbar(label="Temperature")
plt.show()

## Fipy (Finite Volume PDE Solver)

In [ ]:
import fipy as fp

# Problem parameters
L = 1.0  # Length of the domain
alpha = 1.0  # Thermal diffusivity

# Discretization parameters
Nx = 500  # Number of spatial grid points
Nt = 500  # Number of time steps
dx = L / (Nx - 1)
max_time = 0.2  # New maximum time
dt = max_time / (Nt - 1)

# Initial condition
def T_initial(x):
    return np.sin(np.pi * x) + 0.5 * np.sin(3 * np.pi * x) + 0.25 * np.sin(5 * np.pi * x)

# Create the mesh
mesh = fp.Grid1D(nx=Nx, dx=L/Nx)

# Define the temperature variable
T = fp.CellVariable(name="temperature", mesh=mesh, value=0.)

T.setValue(T_initial(T.mesh.x.value))

# Boundary conditions (Dirichlet)
T_left = 0.0
T_right = 0.0

# Set boundary conditions
T.constrain(T_left, where=mesh.facesLeft)
T.constrain(T_right, where=mesh.facesRight)

# Define the transient diffusion equation
eq = fp.TransientTerm() == fp.DiffusionTerm(coeff=alpha)

# Initialize temperature history array
T_history = np.zeros((Nt, Nx))

# Time-stepping
t = 0
for step in range(Nt):
    t += dt
    eq.solve(var=T, dt=dt)
    T_history[step, :] = T.value

In [ ]:
# Create contour plot
x_values = mesh.x.value
t_values = np.linspace(0, max_time, Nt)
X, T_grid = np.meshgrid(x_values, t_values)
plt.contourf(X, T_grid, T_history, cmap="jet")
plt.xlabel("x")
plt.ylabel("t")
plt.colorbar(label="Temperature")
plt.title("Temperature Distribution (Fipy)")
plt.show()

## PINN

In [ ]:
class PINN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_units):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList()
        in_units = input_dim
        for units in hidden_units:
            self.layers.append(nn.Linear(in_units, units))
            in_units = units
        self.layers.append(nn.Linear(in_units, output_dim))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = torch.tanh(layer(x))
        x = self.layers[-1](x)
        return x

In [ ]:
def pinn_loss(model, x, t, alpha, T_left, T_right, T_initial):
    T = model(torch.cat([x, t], dim=1))

    T_x = torch.autograd.grad(T, x, grad_outputs=torch.ones_like(T), create_graph=True, retain_graph=True, only_inputs=True)[0]
    T_xx = torch.autograd.grad(T_x, x, grad_outputs=torch.ones_like(T_x), create_graph=True, retain_graph=True, only_inputs=True)[0]
    T_t = torch.autograd.grad(T, t, grad_outputs=torch.ones_like(T), create_graph=True, retain_graph=True, only_inputs=True)[0]

    f = T_t - alpha * T_xx

    # Initial condition loss
    T_initial_pred = model(torch.cat([x, torch.zeros_like(t)], dim=1))
    T_initial_true = T_initial(x.detach().numpy())  # Detach x before converting to a numpy array
    T_initial_true = torch.tensor(T_initial_true, dtype=torch.float32)  # Convert back to a tensor
    ic_loss = torch.mean(torch.square(T_initial_pred - T_initial_true))

    # Boundary conditions loss
    T_left_pred = model(torch.cat([torch.zeros_like(x), t], dim=1))
    T_right_pred = model(torch.cat([torch.ones_like(x), t], dim=1))
    bc_loss = torch.mean(torch.square(T_left_pred - T_left)) + torch.mean(torch.square(T_right_pred - T_right))

    # Combine PDE residual, initial condition, and boundary condition losses
    total_loss = torch.mean(torch.square(f)) + ic_loss + bc_loss

    return total_loss

In [ ]:
input_dim = 2
output_dim = 1
hidden_units = [64, 64, 64, 64]
alpha = 1.0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pinn = PINN(input_dim, output_dim, hidden_units).to(device)

In [ ]:
learning_rate = 1e-3
epochs = 10000
optimizer = torch.optim.Adam(pinn.parameters(), lr=learning_rate)

for epoch in range(epochs):
    x = torch.tensor(np.random.uniform(0, 1, size=(100, 1)), dtype=torch.float32, device=device, requires_grad=True)
    t = torch.tensor(np.random.uniform(0, 1, size=(100, 1)), dtype=torch.float32, device=device, requires_grad=True)
    
    optimizer.zero_grad()
    loss = pinn_loss(pinn, x, t, alpha, T_left, T_right, T_initial)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

learning_rate = 1e-4
epochs = 10000
optimizer = torch.optim.Adam(pinn.parameters(), lr=learning_rate)

for epoch in range(epochs):
    x = torch.tensor(np.random.uniform(0, 1, size=(100, 1)), dtype=torch.float32, device=device, requires_grad=True)
    t = torch.tensor(np.random.uniform(0, 1, size=(100, 1)), dtype=torch.float32, device=device, requires_grad=True)
    
    optimizer.zero_grad()
    loss = pinn_loss(pinn, x, t, alpha, T_left, T_right, T_initial)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
learning_rate = 1e-5
epochs = 10000
optimizer = torch.optim.Adam(pinn.parameters(), lr=learning_rate)

for epoch in range(epochs):
    x = torch.tensor(np.random.uniform(0, 1, size=(100, 1)), dtype=torch.float32, device=device, requires_grad=True)
    t = torch.tensor(np.random.uniform(0, 1, size=(100, 1)), dtype=torch.float32, device=device, requires_grad=True)
    
    optimizer.zero_grad()
    loss = pinn_loss(pinn, x, t, alpha, T_left, T_right, T_initial)
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [ ]:
def analytical_solution(x, t, alpha):
    term1 = np.sin(np.pi * x) * np.exp(-(np.pi * alpha)**2 * t)
    term2 = 0.5 * np.sin(3 * np.pi * x) * np.exp(-(9 * np.pi * alpha)**2 * t)
    term3 = 0.25 * np.sin(5 * np.pi * x) * np.exp(-(25 * np.pi * alpha)**2 * t)
    
    T = term1 + term2 + term3
    return T

def analytical_solution_2d(x, t, alpha):
    x = np.array(x)
    t = np.array(t)
    
    x_grid, t_grid = np.meshgrid(x, t)

    term1 = np.sin(np.pi * x_grid) * np.exp(-(np.pi * alpha)**2 * t_grid)
    term2 = 0.5 * np.sin(3 * np.pi * x_grid) * np.exp(-(9 * np.pi * alpha)**2 * t_grid)
    term3 = 0.25 * np.sin(5 * np.pi * x_grid) * np.exp(-(25 * np.pi * alpha)**2 * t_grid)

    T = term1 + term2 + term3
    return T


In [ ]:
# Generate a grid of points in the x-t domain
x_test = np.linspace(0, 1, 100)
t_test = np.linspace(0, max_time, 100)
x_grid, t_grid = np.meshgrid(x_test, t_test)
x_grid = x_grid.reshape(-1, 1)
t_grid = t_grid.reshape(-1, 1)

In [ ]:
# T_analytical = analytical_solution(x_grid, t_grid, alpha)
T_analytical = analytical_solution_2d(x_test, t_test, alpha)

In [ ]:
# Evaluate the model at the grid points
x_test_tensor = torch.tensor(x_grid, dtype=torch.float32, device=device, requires_grad=True)
t_test_tensor = torch.tensor(t_grid, dtype=torch.float32, device=device, requires_grad=True)
T_test = pinn(torch.cat([x_test_tensor, t_test_tensor], dim=1))
T_test = T_test.detach().cpu().numpy().reshape(100, 100)

In [ ]:
# Create a contour plot of the temperature distribution
plt.figure()
plt.contourf(x_test, t_test, T_test, cmap="jet")
plt.xlabel("x")
plt.ylabel("t")
plt.title("Temperature Distribution (PINN)")
plt.colorbar(label="Temperature")
plt.show()

# plt.figure()
# plt.contourf(np.linspace(0, L, Nx), np.linspace(0, max_time, Nt), T_matrix.T, cmap="jet")
# plt.xlabel("x")
# plt.ylabel("t")
# plt.title("Temperature Distribution (Crank-Nicolson)")
# plt.colorbar(label="Temperature")
# plt.show()

x_values = mesh.x.value
t_values = np.linspace(0, max_time, Nt)
X, T_grid = np.meshgrid(x_values, t_values)
plt.contourf(X, T_grid, T_history, cmap="jet")
plt.xlabel("x")
plt.ylabel("t")
plt.colorbar(label="Temperature")
plt.title("Temperature Distribution (Fipy)")
plt.show()

plt.figure()
plt.contourf(x_test, t_test, T_analytical, cmap="jet")
plt.xlabel("x")
plt.ylabel("t")
plt.title("Temperature Distribution (Analytical)")
plt.colorbar(label="Temperature")
plt.show()